In [14]:
import sys
import rospy
import moveit_commander
import copy
import moveit_msgs.msg
import geometry_msgs.msg

from math import pi

from tf.transformations import *
from moveit_msgs.msg import RobotState, Constraints, JointConstraint, OrientationConstraint

from std_msgs.msg import String
from moveit_commander.conversions import pose_to_list

In [2]:
rospy.init_node('bf_ur5', anonymous=True)
moveit_commander.roscpp_initialize(sys.argv)

robot = moveit_commander.RobotCommander()
scene = moveit_commander.PlanningSceneInterface()

robot.get_group_names()

['arm', 'gripper']

In [15]:
group_name = robot.get_group_names()
move_group = moveit_commander.MoveGroupCommander(group_name[0])
planning_frame = move_group.get_planning_frame()

In [4]:
eef_link = move_group.get_end_effector_link()

In [5]:
robot_state = robot.get_current_state()
robot_state

joint_state: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: "world"
  name: [shoulder_pan_joint, shoulder_lift_joint, elbow_joint, wrist_1_joint, wrist_2_joint,
  wrist_3_joint, robotiq_85_left_inner_knuckle_joint, robotiq_85_left_finger_tip_joint,
  robotiq_85_left_knuckle_joint, robotiq_85_left_finger_joint, robotiq_85_right_inner_knuckle_joint,
  robotiq_85_right_finger_tip_joint, robotiq_85_right_knuckle_joint, robotiq_85_right_finger_joint]
  position: [-0.7659573740838068, 0.015140461945081896, 3.003261852274619e-05, 0.000387609783984999, 0.00014370332788082862, 0.002287101712152406, 9.870801558342635e-06, -9.870801558342635e-06, 9.870801558342635e-06, 0.0, 9.870801558342635e-06, -9.870801558342635e-06, 9.870801558342635e-06, 0.0]
  velocity: []
  effort: []
multi_dof_joint_state: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: "world"
  joint_names: []
  transforms: []
  twist: []
  wrench: []
attached_

In [17]:

joint_goal = move_group.get_current_joint_values()
joint_goal

[-0.49269514923694846,
 -1.9717276113667417,
 -1.2001757974766738,
 -3.1098775895995585,
 -0.4915896126040904,
 0.007972538585320876]

In [6]:
def get_goal_pose(move_group):
    joint_state = move_group.get_current_pose()
    print joint_state

In [18]:
get_goal_pose(move_group)

header: 
  seq: 0
  stamp: 
    secs: 1895
    nsecs: 661000000
  frame_id: "world"
pose: 
  position: 
    x: -0.439966586816
    y: 0.449936223925
    z: 1.37397680548
  orientation: 
    x: -0.707488601506
    y: -0.706709684523
    z: 0.00301973654571
    w: 0.00349023369959


In [19]:
import geometry_msgs.msg
from geometry_msgs.msg import Quaternion, Point

def bo_move_position( position, orientation ):
    pose_goal = geometry_msgs.msg.Pose()
    pose_goal.orientation = orientation
    pose_goal.position = position
    move_group.set_pose_target(pose_goal)
    plan = move_group.go(wait=True)
    move_group.stop()
    move_group.clear_pose_targets()


In [47]:
orientation_0 = Quaternion(-0.707732860392,
                          -0.706466425662,
                           0.00311631615142,
                           0.00311067637957)

quat = quaternion_from_euler(0, 0, pi/2.0)
ori_0 = Quaternion( quat[0], quat[1], quat[2], quat[3])

quat = quaternion_from_euler(0, 0, -pi/2.0)
ori_1 = Quaternion( quat[0], quat[1], quat[2], quat[3])

Y_ENTRY_POINT = 0.45
Y_INSERT_LENGTH = 0.10


In [48]:
# rack coordinate


rack_0U = [ Point(-0.44, Y_ENTRY_POINT, 0.998+0.376),
           Point(-0.22, Y_ENTRY_POINT, 0.998+0.376),
           Point(0.0, Y_ENTRY_POINT, 0.998+0.376),
           Point(0.22, Y_ENTRY_POINT, 0.998+0.376),
           Point(0.44, Y_ENTRY_POINT, 0.998+0.376)
          ]

rack_0L = [ Point(-0.44, Y_ENTRY_POINT, 0.998-0.376),
           Point(-0.22, Y_ENTRY_POINT, 0.998-0.376),
           Point(0.0, Y_ENTRY_POINT, 0.998-0.376),
           Point(0.22, Y_ENTRY_POINT, 0.998-0.376),
           Point(0.44, Y_ENTRY_POINT, 0.998-0.376)
          ]

rack_1U = [ Point(-0.44, -Y_ENTRY_POINT, 0.998+0.376),
           Point(-0.22, -Y_ENTRY_POINT, 0.998+0.376),
           Point(0.0, -Y_ENTRY_POINT, 0.998+0.376),
           Point(0.22, -Y_ENTRY_POINT, 0.998+0.376),
           Point(0.44, -Y_ENTRY_POINT, 0.998+0.376)
          ]

rack_1L = [ Point(-0.44, -Y_ENTRY_POINT, 0.998-0.376),
           Point(-0.22, -Y_ENTRY_POINT, 0.998-0.376),
           Point(0.0, -Y_ENTRY_POINT, 0.998-0.376),
           Point(0.22, -Y_ENTRY_POINT, 0.998-0.376),
           Point(0.44, -Y_ENTRY_POINT, 0.998-0.376)
          ]


In [49]:
# go to the initial position 
bo_move_position(rack_0U[0])

for point in rack_0U:
    bo_move_position(point, ori_0)
    bo_move_position(Point(point.x, point.y+Y_INSERT_LENGTH, point.z), ori_0)
    bo_move_position(point, ori_0 )

for point in reversed(rack_0L):
    bo_move_position(point, ori_0)
    bo_move_position(Point(point.x, point.y+Y_INSERT_LENGTH, point.z), ori_0)
    bo_move_position(point, ori_0)
    
# when we go to the opposite side of the rack, orientation should be reconsidered.
    
for point in rack_1U:
    bo_move_position(point, ori_1)
    bo_move_position(Point(point.x, point.y-Y_INSERT_LENGTH, point.z), ori_1)
    bo_move_position(point, ori_1)

for point in reversed(rack_1L):
    bo_move_position(point, ori_1)
    bo_move_position(Point(point.x, point.y-Y_INSERT_LENGTH, point.z), ori_1)
    bo_move_position(point, ori_1)

# go to the initial position
bo_move_position(rack_0U[0])


In [9]:
constraints = Constraints()

constraints.name = "constraints1"

oc1 = OrientationConstraint()

# input orientation constraints
constraints.orientation_constraints.append(oc1)
move_group.set_path_constraints(constraints)


In [10]:
from geometry_msgs.msg import Quaternion

In [11]:
a = Quaternion(0,0,0,1)

In [12]:
a

x: 0
y: 0
z: 0
w: 1